In [19]:
%%html

<style>    
    @import url("css/custom_styles.css")
</style>

In [70]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

#
# import utilitaires
#
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import helpers as hlp
import helpers.dataset.PlantVillage as pv
import helpers.features as feat

In [21]:
#
# parametres dataset PlantVillage
#
config_overrides = hlp.MetaObject.from_json("config_overrides.json")
executor = hlp.create_thread_pool_executor(max_workers=None)

pv_config = pv.Config(executor)
if not config_overrides is None:
    hlp.MetaObject.override_from_object(pv_config, 
                                        config_overrides.dataset)

print("pv_config")
display(vars(pv_config))

#
# obtenir le dataset PlantVillage
#
with hlp.Profile() as pv_loading:
    pv_dataset = pv.load(pv_config)

    if pv_dataset is None:
        print("Invalid dataset")
    else:
        pv_dataframe = pv_dataset.dataframe

print(f"Loading dataset: {round(pv_loading.duration, 2)}s")
print("PlantVillage", pv_dataframe.shape)

pv_config


{'url': 'https://tinyurl.com/22tas3na',
 'install_path': 'dataset/PlantVillage.hd5',
 'species_disease_re': '(.*)(?:___)(.*)',
 'species_re': '(.*)(?:,_|_)(.*)',
 'label_separator': '_',
 'thumbnail_scale': 0.25,
 'force_download': False,
 'read_only': True,
 'executor': <concurrent.futures.thread.ThreadPoolExecutor at 0x1df4db38400>,
 'force_install': False}

Loading dataset: 0.6s
PlantVillage (60343, 5)


In [22]:
df = pv_dataset.dataframe[["label", "species"]]

In [18]:
nbLabel = df.groupby("label").count()
#print(nbLabel)

In [23]:
nbEspece = df.groupby("species").count()
print(nbEspece)

              label
species            
Apple          4645
Blueberry      1502
Cherry         2052
Corn           4354
Grape          4639
Orange         5507
Peach          3297
Pepper,_bell   2478
Potato         3000
Raspberry      1000
Soybean        5090
Squash         1835
Strawberry     2109
Tomato        18835


In [54]:
healthy_crit = df["label"].str.contains("healthy")
healthy = df[healthy_crit]
healthy = healthy.groupby("species").count()

not_healthy = df[~healthy_crit]
not_healthy = not_healthy.groupby("species").count()
print('Healthy:\n',healthy)
print('Not healthy:\n',not_healthy)

Healthy:
               label
species            
Apple          1645
Blueberry      1502
Cherry         1000
Corn           1162
Grape          1000
Peach          1000
Pepper,_bell   1478
Potato         1000
Raspberry      1000
Soybean        5090
Strawberry     1000
Tomato         1591
Not healthy:
               label
species            
Apple          3000
Cherry         1052
Corn           3192
Grape          3639
Orange         5507
Peach          2297
Pepper,_bell   1000
Potato         2000
Squash         1835
Strawberry     1109
Tomato        17244


On enlève donc Blueberry, Soybean, Rasberry, Squash et Orange parce qu'ils ne sont pas représenté dans les deux catégories (healthy et not healthy).

In [76]:
df1 = pv_dataset.dataframe

searchfor = ["Apple", "Cherry", "Corn", "Grape", "Peach", "Pepper,_bell", "Potato", "Strawberry", "Tomato"]
group_sel = df1[df1["label"].str.contains('|'.join(searchfor))]
# in 
t = group_sel.groupby("species").count()

print(t)

              disease  label  image_path  thumbnail_path
species                                                 
Apple            4645   4645        4645            4645
Cherry           2052   2052        2052            2052
Corn             4354   4354        4354            4354
Grape            4639   4639        4639            4639
Peach            3297   3297        3297            3297
Pepper,_bell     2478   2478        2478            2478
Potato           3000   3000        3000            3000
Strawberry       2109   2109        2109            2109
Tomato          18835  18835       18835           18835


In [55]:
tomato_crit = df["label"].str.contains("Tomato")
tomato = df[tomato_crit]

tomato_count = tomato["label"].value_counts()
tomato_count.sort_values

print(tomato_count)


Tomato_Yellow_Leaf_Curl_Virus                  5357
Tomato_Bacterial_spot                          2127
Tomato_Late_blight                             1909
Tomato_Septoria_leaf_spot                      1771
Tomato_Spider_mites Two-spotted_spider_mite    1676
Tomato_healthy                                 1591
Tomato_Target_Spot                             1404
Tomato_Early_blight                            1000
Tomato_Leaf_Mold                               1000
Tomato_mosaic_virus                            1000
Name: label, dtype: int64


In [69]:
df1.head()

,species,disease,label,image_path,thumbnail_path
0,Apple,Apple_scab,Apple_scab,Plant_leave_diseases_dataset_with_augmentation...,thumbnails/Plant_leave_diseases_dataset_with_a...
1,Apple,Apple_scab,Apple_scab,Plant_leave_diseases_dataset_with_augmentation...,thumbnails/Plant_leave_diseases_dataset_with_a...
2,Apple,Apple_scab,Apple_scab,Plant_leave_diseases_dataset_with_augmentation...,thumbnails/Plant_leave_diseases_dataset_with_a...
3,Apple,Apple_scab,Apple_scab,Plant_leave_diseases_dataset_with_augmentation...,thumbnails/Plant_leave_diseases_dataset_with_a...
4,Apple,Apple_scab,Apple_scab,Plant_leave_diseases_dataset_with_augmentation...,thumbnails/Plant_leave_diseases_dataset_with_a...


In [84]:
# *** À déterminer les prédicteurs x et la variable cible car ce n'est pas ca j'ai mis ca en attendant 


x = group_sel.drop("label", axis=1)
y = group_sel["label"]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, stratify=y, test_size=0.2, random_state=44)